Import TensorFlow and other necessary libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import glob

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model

Test GPU support

In [ ]:
import tensorflow as tf 

if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

tf.config.list_physical_devices()

# Get data, training and evaluating 

In [ ]:
BASE_DIR_WITH_DATA = 'data/annotated_images/segmented_frames'

def get_data_augmentation_layers(img_size):
    return keras.Sequential(
      [
        layers.RandomFlip("horizontal", input_shape=(img_size[0], img_size[1], 3)),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.2),
      ]
    )

def normalize_and_prefetch_data(train_ds, val_ds, data_augmentation_layers=None):
    #normalize data
    normalization_layer = layers.Rescaling(1./255)
    train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
    print('normalized')

    #cache data
    train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    print('cached')

    if data_augmentation_layers is not None:
        train_ds = train_ds.map(lambda x, y: (data_augmentation_layers(x), y))
        print('augumented train ds')

    return train_ds, val_ds

def get_data(img_size=(64, 64), augment_data_during_loading=False):
    print(f'get data for approach: {approach_name}')
    seed = np.random.randint(1e6)
    data_dir = pathlib.Path(f'{BASE_DIR_WITH_DATA}/{approach_name}')

    train_ds = tf.keras.utils.image_dataset_from_directory(
      data_dir,
      validation_split=0.2,
      subset="training",
      shuffle=True,
      seed=seed,
      image_size=img_size,
      batch_size=batch_size)

    val_ds = tf.keras.utils.image_dataset_from_directory(
      data_dir,
      validation_split=0.2,
      subset="validation",
      shuffle=True,
      seed=seed,
      image_size=img_size,
      batch_size=batch_size)

    if augment_data_during_loading:
        return normalize_and_prefetch_data(train_ds, val_ds, get_data_augmentation_layers(img_size))

    return normalize_and_prefetch_data(train_ds, val_ds)

def get_model(type='cnn', img_size=(64, 64), num_classes=2):
    data_augmentation = get_data_augmentation_layers(img_size)

    if type == 'cnn':
        model = Sequential([
            data_augmentation,
            layers.Conv2D(int(img_size[0]/2), (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(img_size[0], (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(img_size[0], (3, 3), activation='relu'),
            layers.Dropout(0.2),
            layers.Flatten(),
            layers.Dense(img_size[0], activation='relu'),
            layers.Dense(1, activation='sigmoid', name='outputs')
        ])
    elif type == 'vgg16':
        base_model = tf.keras.applications.vgg16.VGG16(
            include_top=False,
            weights=None,
            input_shape=(img_size[0], img_size[1], 3),
        )
        x = layers.GlobalAveragePooling2D()(base_model.output)
        predictions = layers.Dense(1, activation='sigmoid')(x) # Add a fully connected layer with a sigmoid activation for binary classification
        model = Model(inputs=base_model.input, outputs=predictions) # Create the final model
    elif type == 'inception':
        base_model = tf.keras.applications.inception_v3.InceptionV3(
            include_top=False,
            weights=None,
            input_shape=(img_size[0], img_size[1], 3),
        )
        x = layers.GlobalAveragePooling2D()(base_model.output)
        predictions = layers.Dense(1, activation='sigmoid')(x) # Add a fully connected layer with a sigmoid activation for binary classification
        model = Model(inputs=base_model.input, outputs=predictions) # Create the final model
    elif type == 'resnet':
        base_model = tf.keras.applications.ResNet50(
            include_top=False,
            weights=None,
            input_shape=(img_size[0], img_size[1], 3),
        )

        x = layers.GlobalAveragePooling2D()(base_model.output)
        predictions = layers.Dense(1, activation='sigmoid')(x) # Add a fully connected layer with a sigmoid activation for binary classification
        model = Model(inputs=base_model.input, outputs=predictions) # Create the final model

    else:
        raise Exception('wrong type of model')
    
    return model

def train_model(train_ds, val_ds, epochs = 30, img_size=(64, 64), num_classes=2, model_type='cnn', model=None):
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', verbose=1, patience=3)

    if model is None:
        model = get_model(type=model_type, img_size=img_size, num_classes=num_classes)

        #compile and summary model
        model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
        model.summary()

    #compute class weights
    data_dir = pathlib.Path(f'{BASE_DIR_WITH_DATA}/{approach_name}')
    total = len(list(glob.glob(f'{data_dir}/*/*.jpg')))
    neg = len(list(glob.glob(f'{data_dir}/punch/*.jpg')))
    pos = len(list(glob.glob(f'{data_dir}/not_punch/*.jpg')))
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)
    class_weight = {0: weight_for_0, 1: weight_for_1}
    print('Weight for class 0: {:.2f}'.format(weight_for_0))
    print('Weight for class 1: {:.2f}'.format(weight_for_1))

    #train model
    history = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=epochs,
      class_weight=class_weight,
      callbacks=[early_stopping_callback]
    )

    return history, model

def evaluate_model(val_ds, model):
    y_pred = []
    y_true = []

    for batch_images, batch_labels in val_ds:
        predictions = model.predict(batch_images, verbose=0)
        y_pred = y_pred + np.argmax(tf.nn.softmax(predictions), axis=1).tolist()
        y_true = y_true + batch_labels.numpy().tolist()
    print(classification_report(y_true, y_pred))
    # print(confusion_matrix(y_true, y_pred))
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn, fp, fn, tp

def evaluate_binary_model(val_ds, model):
    y_pred = []
    y_true = []

    for batch_images, batch_labels in val_ds:
        predictions = model.predict(batch_images, verbose=0)
        y_pred_batch = (predictions > 0.5).astype('int32').flatten()
        y_pred = y_pred + y_pred_batch.tolist()
        y_true = y_true + batch_labels.numpy().tolist()
    print(classification_report(y_true, y_pred))

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn, fp, fn, tp

def visualize_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs_range = range(len(acc))
    
    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
    
def train_and_evaluate(img_size, model_type):
    train_ds, val_ds = get_data(img_size=img_size, augment_data_during_loading=model_type != 'cnn')
    history, model = train_model(train_ds, val_ds, epochs=epochs, img_size=img_size, model_type=model_type)
    tn, fp, fn, tp = evaluate_binary_model(val_ds, model)
    
    return history

## To reproduce
1. Run above cell with function definitions.
2. Define the approach in which you want to train classifier by setting value on `approach_name` variable.
Possible values of `approach_name` variable: `original`, `extract_colours`, `background_subtraction_by_mog2`, `background_subtraction_by_knn`, `hybrid_extraction` and `speed_movement_extraction_{compare_with_n_back_frame}` (where `compare_with_n_back_frame` is variable to set).
3. Run above cell to start training and evaluating classifier.

**Remember**: in order to train classifier in a specific approach, you are need to prepare dataset for this approach using `preprocess_frames_before_classification.py` algorithm.

PS you can try to train the classifier on another type of model, to do this change `model_type` parameter, possible values: `cnn`(custom structure) `vgg16`, `inception`(inceptionv3), `resnet`(resnet50). 

In [ ]:
epochs = 80
batch_size = 512
approach_name = 'original'

training_history = train_and_evaluate(img_size=(80, 80), model_type='cnn')
visualize_training_history(training_history)